In [1]:
from access_data import *
from preparing_data import *
from evaluating_data import *
# from feature_engineering import *

C:\Users\Max\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import numpy as np
from sklearn.metrics import accuracy_score

#getting all data
datasets = get_df_map()
thresholdMap = {}

for key, df in datasets.items():
    #splitting data
    split_map_all = split_dataset(df, validate_set=True)
    X_train_all = split_map_all['X_train']
    y_train_all = split_map_all['y_train']
    X_validate_all = split_map_all['X_validate']
    y_validate_all = split_map_all['y_validate']
    X_test_all = split_map_all['X_test']
    y_test_all = split_map_all['y_test']

    #encoding label
    integer_encoded_train = y_train_all.replace("NONE", 0).replace(key, 1)
    integer_encoded_validate = y_validate_all.replace("NONE", 0).replace(key, 1)
    integer_encoded_test = y_test_all.replace("NONE", 0).replace(key, 1)
    
    tfidf_vectorizer =  TfidfVectorizer(stop_words='english', max_df=0.15, min_df=5)
    X_train = tfidf_vectorizer.fit_transform(X_train_all)
    X_validate = tfidf_vectorizer.transform(X_validate_all)
    X_test = tfidf_vectorizer.transform(X_test_all)

    logReg = LogisticRegression(C=5)
    
    logReg.fit(X_train, integer_encoded_train)
    # predict probabilities
    probs = logReg.predict_proba(X_validate)
    predictions = convert_probability_list(probs)
    prob_baseline = convert_probability_trueFalse(predictions, 0.5)
    
    # prediction on test set
    probs_test = logReg.predict_proba(X_test)
    predictions_test = convert_probability_list(probs_test)
    
    
    thresholds = {}
    for i in [0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 
              0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:        
        predictions_binary = convert_probability_trueFalse(predictions, i)
        binary = convert_probability_trueFalse(predictions_test, 0.5)
        
        acc = accuracy_score(integer_encoded_test,binary)
        f1_score_micro = f1_score(integer_encoded_validate, predictions_binary, average='micro')        
        print('############## F1 score for label {} with thrashold {}: {} ###############'.format(key, i, f1_score_micro))
        thresholds[i] = f1_score_micro
       
    thresh = 0
    best_f1 = 0
    for key, value in thresholds.items():        
        if best_f1 <= value:
            best_f1 = value
            thresh = key
    
    
    print('###################### {} ####################'.format(key))
    print('--------------------------baseline------------------------') 
    report = classification_report(integer_encoded_validate, prob_baseline)
    print(report)
    print('--------------------adjusted threshold {}-------------------'.format(thresh))
    prediction_bin_test = convert_probability_trueFalse(predictions_test, thresh)
    report_test = classification_report(integer_encoded_test, prediction_bin_test)
    print(report_test)

There were not enough data to create a split with split_ratio 0.3. Only selected 70164 instead of 136020 random data
There were not enough data to create a split with split_ratio 0.3. Only selected 76675 instead of 114317 random data
############## F1 score for label ENTERTAINMENT with thrashold 0.05: 0.9489501667117239 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.1: 0.9490853383797423 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.15: 0.9443543299990989 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.2: 0.9358385149139405 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.25: 0.9256105253672164 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.3: 0.9152473641524737 ###############
############## F1 score for label ENTERTAINMENT with thrashold 0.35: 0.8991168784356133 ###############
############## F1 score for label ENTERTA